In [1]:
# import modules
import xlsxwriter
import random
import pandas as pd
import numpy as np

In [49]:
class DataFrameExcelCharting(object):
    def __init__(self, df):
        self.data = df
        self.num_rows = len(df)
        self.column_map = dict()
        self._to_excel = 0
        
    def createWorkBook(self, workbook_name):
        """create work book"""
        self.workbook_name = workbook_name
        self.workbook = xlsxwriter.Workbook(self.workbook_name)
        
    def createWorkSheet(self, worksheet_name):
        """create work sheet in self.workbook"""
        self.worksheet_name = worksheet_name
        self.worksheet = self.workbook.add_worksheet(self.worksheet_name)
        
    def closeWorkBook(self):
        """close work book"""
        self.workbook.close()
        
    def createChart(self, chart_type, x_axis, y_axis, title):
        """create chart"""
        self.chart = self.workbook.add_chart({'type': chart_type})
        self.chart.set_y_axis({'name': y_axis})
        self.chart.set_x_axis({'name': x_axis})
        self.chart.set_title({'name': title})
        
    def insertChart(self, insert_col, insert_row):
        """insert chart to work sheet"""
        self.worksheet.insert_chart('{0}{1}'.format(insert_col, insert_row), self.chart)
        
    def writeToExcel(self, file_name="test_file.xlsx", sheet_name="Sheet1"):
        """write data frame to excel with header"""
        # create workbook and worksheet
        self.createWorkBook(file_name)
        self.createWorkSheet(sheet_name)
        # write data frame to excel
        header_row = 1
        for i in range(0, len(self.data.columns)):
            col = xlsxwriter.utility.xl_col_to_name(i)
            self.worksheet.write_column("{0}{1}".format(col, header_row), self.data.columns[i])
            self.worksheet.write_column("{0}{1}".format(col, header_row + 1), self.data[self.data.columns[i]].values)
            self.column_map[self.data.columns[i]] = col
        # change the flag
        self._to_excel = 1
            
    def topNChart(self, columns=None, n=5, category_col=None, 
                  chart_type="column", x_axis="name", y_axis="value", title="title"):
        """top N chart"""
        if not self._to_excel:
            print "Please write data to excel first"
            return
        
        if columns is None:
            print "Please specify a list of columns to plot"
            return
        elif not isinstance(columns, list):
            columns = list(columns)
        
        if category_col:
            cat_col = self.column_map[category_col]
            
        # create a chart
        self.createChart(chart_type, x_axis, y_axis, title)
        
        for i in range(0, len(columns)):
            col = self.column_map[columns[i]]
            self.chart.add_series({'values': "={0}!${1}$2:${1}${2}".format(self.worksheet_name, col, n), 
                                   'name': columns[i], 
                                   'categories': "={0}!${1}$2:${1}${2}".format(self.worksheet_name, cat_col, n)
                                  })
        # insert the chart    
        self.insertChart(col, self.num_rows + 2)
    
    def getBucketsCounts(self, column=None, n_buckets=5):
        """get buckets and counts"""
        # None column check
        if column is None:
            print "Please specify a column name "
            return
        # calculate count and interval
        lower = np.floor(self.data[column].min())
        upper = np.ceil(self.data[column].max())
        diff = upper - lower
        bins = [np.arange(lower, upper, diff / n_buckets)]
        bins = np.append(bins, upper)
        count, interval = np.histogram(self.data[column], bins=bins)
        interval = ["[{0}, {1})".format(interval[i], interval[i + 1]) for i in range(0,len(interval) - 1)] # + diff / n_buckets / 2.0 
        return count, interval
            
    def bucketsNChart(self, column=None, n_buckets=5,
                      chart_type="column", x_axis="name", y_axis="value", title="title"):
        """top N chart"""
        if not self._to_excel:
            print "Please write data to excel first"
            return
        # create a new chart
        self.createChart(chart_type, x_axis, y_axis, title)
        
        """plot bucket chart"""
        count, interval = self.getBucketsCounts(column=column, n_buckets=n_buckets)
        col = self.column_map[column]
        row = self.num_rows + 3
        self.worksheet.write_column('{0}{1}'.format(col, row), count)
        self.worksheet.write_column('{0}{1}'.format(col, row + n_buckets), interval)
        
        self.chart.add_series({'values': '={0}!${1}${2}:${1}${3}'.format(self.worksheet_name, col, row, row + n_buckets - 1),
                               'categories': '={0}!${1}${2}:${1}${3}'.format(self.worksheet_name, col, row + n_buckets, row + n_buckets + n_buckets -1),
                               'gap': 5
                              })
        # insert chart
        self.insertChart(col, row)

In [52]:
# test data frame
df = pd.DataFrame(np.random.randn(50, 4), columns=list('ABCD'))
df["category"] = ["category{}".format(i) for i in range(50)]
print df.head()

          A         B         C         D   category
0 -1.465227  0.577587  1.356299 -0.340890  category0
1  0.126552  0.894140 -2.453407  0.190501  category1
2 -0.500601 -2.218207 -1.627706 -0.735097  category2
3  0.565776 -0.216808 -0.818934  1.484259  category3
4  0.890662 -0.544240 -1.802556  0.909245  category4


In [ ]:
sales = [('Jones LLC', 150, 200, 50),
         ('Alpha Co', 200, 210, 90),
         ('Blue Inc', 140, 215, 95)]
labels = ['account', 'Jan', 'Feb', 'Mar']
df = pd.DataFrame.from_records(sales, columns=labels)

In [53]:
test_class = DataFrameExcelCharting(df)
test_class.writeToExcel("test_top_N.xlsx", "test_sheet")
test_class.topNChart(columns=["A", "C"], n=10, category_col="category")
test_class.bucketsNChart(column="A",n_buckets=10)
test_class.closeWorkBook()

In [6]:

# sample use case
top_n_chart(df, 
            columns=["A", "B", "C"], 
            n=10, 
            category="category", 
            file_name="test_top_N.xlsx", 
            x_axis="Category", 
            y_axis="Value", 
            title="Top N of Category")

# sample use case
n_buckets_chart(df, 
                column="A", 
                n_buckets=10, 
                file_name="test_n_buckets.xlsx", 
                x_axis="Interval", 
                y_axis="Count", 
                title="N Buckets of A")

In [11]:
# top N bar plot
def top_n_chart(df, columns=None, n=5, category_col=None, file_name="test_file.xlsx", sheet_name="Sheet1", x_axis="name", y_axis="value", title="title"):
    """
    params:
    
    df: pandas dataframe
    columns: list of columns to plot
    n: number of top to plot
    category: string of categorical column name
    """
    # Create workbook and add worksheet
    workbook = xlsxwriter.Workbook(file_name)
    worksheet = workbook.add_worksheet(sheet_name)
    
    # Create a new Chart object.
    chart_type="column"
    chart = workbook.add_chart({'type': chart_type})
    chart.set_y_axis({'name': y_axis})
    chart.set_x_axis({'name': x_axis})
    chart.set_title({'name': title})
    
    # convert single column to list
    if not isinstance(columns, list):
        columns = list(columns)
    
    # Write some data to add to plot on the chart.
    if columns:
        data = df[columns]
    else:
        data = df
    
    for i in range(0, len(data.columns)):
        col = xlsxwriter.utility.xl_col_to_name(i)
        worksheet.write_column("{0}1".format(col), data[data.columns[i]].values)
        if data.columns[i] == category:
            continue
        # Configure the chart. In simplest case we add one or more data series.
        if category:
            chart.add_series({'values': "={}!${0}$1:${0}${1}".format(col, n), 
                              'name': data.columns[i], 
                              'categories': "=Sheet1!$A$1:$A${0}".format(n)})
        else:
            chart.add_series({'values': "=Sheet1!${0}$1:${0}${1}".format(col, n), 
                              'name': data.columns[i]})
    
    # Insert the chart into the worksheet.
    worksheet.insert_chart('{0}{1}'.format(xlsxwriter.utility.xl_col_to_name(i + 1), n + 2), chart)

    workbook.close()

In [3]:
# bucket plot
def n_buckets_chart(df, column=None, n_buckets=5, file_name="test_file.xlsx", x_axis="value", y_axis="count", title="title"):
    """
    params:
    
    df: pandas dataframe
    column: string of column name to bucketize
    n_buckets: number of buckets
    """
    # Create workbook and add worksheet
    workbook = xlsxwriter.Workbook(file_name)
    worksheet = workbook.add_worksheet()
    
    # Create a new Chart object.
    chart_type="column"
    chart = workbook.add_chart({'type': chart_type})
    chart.set_y_axis({'name': y_axis})
    chart.set_x_axis({'name': x_axis})
    chart.set_title({'name': title})
    
    # generate buckets and counts
    lower = np.floor(df[column].min())
    upper = np.ceil(df[column].max())
    diff = upper - lower
    bins = [np.arange(lower, upper, diff / n_buckets)]
    bins = np.append(bins, upper)
    count, interval = np.histogram(df[column], bins=bins)
    interval = ["{0} to {1}".format(interval[i], interval[i + 1]) for i in range(0,len(interval) - 1)] # + diff / n_buckets / 2.0 
    
    # insert columns to xlsx
    worksheet.write_column('A1', count)
    worksheet.write_column('B1', interval)


    # Configure the chart. In simplest case we add one or more data series.
    chart.add_series({'values': '=Sheet1!$A$1:$A${0}'.format(n_buckets),
                      'categories': '=Sheet1!$B$1:$B${0}'.format(n_buckets),
                      'gap': 5
                     })

    # Insert the chart into the worksheet.
    worksheet.insert_chart('{0}{1}'.format("A", n_buckets + 2), chart)

    workbook.close()